In [1]:
import wfdb
import matplotlib.pyplot as plt
import matplotlib 
import pandas as pd
import os
import json
import tqdm
import numpy as np
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn

In [2]:
np.set_printoptions(threshold=np.inf)

In [3]:
pd.set_option("display.max_rows", None)

In [4]:
pd.set_option("display.max_columns", None)

path_data_csv = r"C:\Users\magra\Documents\HSD\5_Semester\Advances_AI\Project\ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.3\ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.3"
#path_data_csv = r"C:\Users\magra\Documents\5_Semester\Advances_AI\ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.3\ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.3"
#path_data_csv =r"/home/miriam/ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.3"

ptbxl = pd.read_csv(os.path.join(path_data_csv, "ptbxl_database.csv"))
ptbxl.head()

,ecg_id,patient_id,age,sex,height,weight,nurse,site,device,recording_date,report,scp_codes,heart_axis,infarction_stadium1,infarction_stadium2,validated_by,second_opinion,initial_autogenerated_report,validated_by_human,baseline_drift,static_noise,burst_noise,electrodes_problems,extra_beats,pacemaker,strat_fold,filename_lr,filename_hr
0,1,15709.0,56.0,1,NaN,63.0,2.0,0.0,CS-12 E,1984-11-09 09:17:34,sinusrhythmus periphere niederspannung,"{'NORM': 100.0, 'LVOLT': 0.0, 'SR': 0.0}",NaN,NaN,NaN,NaN,False,False,True,NaN,", I-V1,",NaN,NaN,NaN,NaN,3,records100/00000/00001_lr,records500/00000/00001_hr
1,2,13243.0,19.0,0,NaN,70.0,2.0,0.0,CS-12 E,1984-11-14 12:55:37,sinusbradykardie sonst normales ekg,"{'NORM': 80.0, 'SBRAD': 0.0}",NaN,NaN,NaN,NaN,False,False,True,NaN,NaN,NaN,NaN,NaN,NaN,2,records100/00000/00002_lr,records500/00000/00002_hr
2,3,20372.0,37.0,1,NaN,69.0,2.0,0.0,CS-12 E,1984-11-15 12:49:10,sinusrhythmus normales ekg,"{'NORM': 100.0, 'SR': 0.0}",NaN,NaN,NaN,NaN,False,False,True,NaN,NaN,NaN,NaN,NaN,NaN,5,records100/00000/00003_lr,records500/00000/00003_hr
3,4,17014.0,24.0,0,NaN,82.0,2.0,0.0,CS-12 E,1984-11-15 13:44:57,sinusrhythmus normales ekg,"{'NORM': 100.0, 'SR': 0.0}",NaN,NaN,NaN,NaN,False,False,True,", II,III,AVF",NaN,NaN,NaN,NaN,NaN,3,records100/00000/00004_lr,records500/00000/00004_hr
4,5,17448.0,19.0,1,NaN,70.0,2.0,0.0,CS-12 E,1984-11-17 10:43:15,sinusrhythmus normales ekg,"{'NORM': 100.0, 'SR': 0.0}",NaN,NaN,NaN,NaN,False,False,True,", III,AVR,AVF",NaN,NaN,NaN,NaN,NaN,4,records100/00000/00005_lr,records500/00000/00005_hr


In [5]:
scp_codes = pd.read_csv(os.path.join(path_data_csv, "scp_statements.csv"))
scp_codes.head()

,Unnamed: 0,description,diagnostic,form,rhythm,diagnostic_class,diagnostic_subclass,Statement Category,SCP-ECG Statement Description,AHA code,aECG REFID,CDISC Code,DICOM Code
0,NDT,non-diagnostic T abnormalities,1.0,1.0,NaN,STTC,STTC,other ST-T descriptive statements,non-diagnostic T abnormalities,NaN,NaN,NaN,NaN
1,NST_,non-specific ST changes,1.0,1.0,NaN,STTC,NST_,Basic roots for coding ST-T changes and abnorm...,non-specific ST changes,145.0,MDC_ECG_RHY_STHILOST,NaN,NaN
2,DIG,digitalis-effect,1.0,1.0,NaN,STTC,STTC,other ST-T descriptive statements,suggests digitalis-effect,205.0,NaN,NaN,NaN
3,LNGQT,long QT-interval,1.0,1.0,NaN,STTC,STTC,other ST-T descriptive statements,long QT-interval,148.0,NaN,NaN,NaN
4,NORM,normal ECG,1.0,NaN,NaN,NORM,NORM,Normal/abnormal,normal ECG,1.0,NaN,NaN,F-000B7


In [ ]:
print(scp_codes["diagnostic_subclass"].unique())
print(scp_codes["diagnostic_class"].unique())
print(scp_codes["Unnamed: 0"].unique())
print(scp_codes.shape)


In [6]:
ptbxl_ann = ptbxl[["ecg_id", "patient_id", "scp_codes", "filename_lr"]]
ptbxl_ann.head()

,ecg_id,patient_id,scp_codes,filename_lr
0,1,15709.0,"{'NORM': 100.0, 'LVOLT': 0.0, 'SR': 0.0}",records100/00000/00001_lr
1,2,13243.0,"{'NORM': 80.0, 'SBRAD': 0.0}",records100/00000/00002_lr
2,3,20372.0,"{'NORM': 100.0, 'SR': 0.0}",records100/00000/00003_lr
3,4,17014.0,"{'NORM': 100.0, 'SR': 0.0}",records100/00000/00004_lr
4,5,17448.0,"{'NORM': 100.0, 'SR': 0.0}",records100/00000/00005_lr


In [7]:
intermediary_store=[]
import json
for index, row in ptbxl_ann.iterrows():
    row["scp_codes"] = row["scp_codes"].replace("'", '"')
    row["scp_codes"] = json.loads(row["scp_codes"])
    intermediary_store.append(row["scp_codes"])
intermediary_store

[{'NORM': 100.0, 'LVOLT': 0.0, 'SR': 0.0},
 {'NORM': 80.0, 'SBRAD': 0.0},
 {'NORM': 100.0, 'SR': 0.0},
 {'NORM': 100.0, 'SR': 0.0},
 {'NORM': 100.0, 'SR': 0.0},
 {'NORM': 100.0, 'SR': 0.0},
 {'NORM': 100.0, 'SR': 0.0},
 {'IMI': 35.0, 'ABQRS': 0.0, 'SR': 0.0},
 {'NORM': 100.0, 'SR': 0.0},
 {'NORM': 100.0, 'SR': 0.0},
 {'NORM': 80.0, 'SARRH': 0.0},
 {'NORM': 80.0, 'SBRAD': 0.0},
 {'NORM': 100.0, 'SR': 0.0},
 {'NORM': 100.0, 'SR': 0.0},
 {'NORM': 100.0, 'SARRH': 0.0},
 {'NORM': 100.0, 'SR': 0.0},
 {'AFLT': 100.0, 'ABQRS': 0.0, 'AFIB': 0.0},
 {'AFLT': 100.0},
 {'NORM': 100.0, 'SR': 0.0},
 {'AFLT': 100.0, 'ABQRS': 0.0},
 {'NORM': 100.0, 'SR': 0.0},
 {'NDT': 100.0, 'SR': 0.0},
 {'AFLT': 100.0},
 {'NORM': 100.0, 'SR': 0.0},
 {'NORM': 100.0, 'SR': 0.0},
 {'NST_': 100.0, 'DIG': 100.0, 'SR': 0.0},
 {'NORM': 80.0, 'SR': 0.0},
 {'NST_': 100.0, 'DIG': 100.0, 'SR': 0.0},
 {'NORM': 100.0, 'SR': 0.0},
 {'LVH': 100.0, 'SR': 0.0},
 {'NORM': 100.0, 'SR': 0.0},
 {'LPFB': 100.0, 'SR': 0.0},
 {'NORM': 100.0

In [8]:
ptbxl_ann["scp_codes"] = intermediary_store

C:\Users\magra\AppData\Local\Temp\ipykernel_14056\1873181576.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ptbxl_ann["scp_codes"] = intermediary_store


In [9]:
data_one_scp_code = []
for index, row in ptbxl_ann.iterrows():
    dictionary = row["scp_codes"]
    
    
    if isinstance(dictionary, dict):
        #print(dictionary)
        # find key that corresponds to highest value
        max_key = max(dictionary, key = dictionary.get)
        #print(max_key)
        # get value associated with max_key
        max_value = dictionary[max_key]

        if max_value >= 70:
            # first two columns stay the same
            ecg_id_value = row["ecg_id"]
            patient_id_value = row["patient_id"]
            filename_100 = row["filename_lr"]
            # add to list for storage
            data_one_scp_code.append([ecg_id_value, patient_id_value, max_key, max_value, filename_100])

data_one_scp_df = pd.DataFrame(data_one_scp_code, columns=["ecg_id", "patient_id", "max_key", "max_value", "filename_lr"])

In [10]:
diagnostic_class = []
diagnostic_subclass = []
for i in data_one_scp_df["max_key"]:
    #print("i", i)
    diagnostic_class.append(scp_codes.loc[scp_codes["Unnamed: 0"] == i, "diagnostic_class"].values[0])
    diagnostic_subclass.append(scp_codes.loc[scp_codes["Unnamed: 0"] == i, "diagnostic_subclass"].values[0])
    

In [11]:
data_one_scp_df["diagnostic_class"] = diagnostic_class
data_one_scp_df["diagnostic_subclass"] = diagnostic_subclass

In [12]:
data_one_scp_df[data_one_scp_df["diagnostic_class"].isnull()].shape 

(651, 7)

In [13]:
data_one_scp_df[data_one_scp_df["diagnostic_subclass"].isnull()].shape

(651, 7)

In [14]:
data_one_scp_df.shape

(19683, 7)

In [15]:
data_one_scp_df=data_one_scp_df.dropna()

In [16]:
data_one_scp_df.shape

(19032, 7)

In [17]:
data_one_scp_df["diagnostic_class_numerical"] = data_one_scp_df.loc[:, "diagnostic_class"]
data_one_scp_df.head(5)

,ecg_id,patient_id,max_key,max_value,filename_lr,diagnostic_class,diagnostic_subclass,diagnostic_class_numerical
0,1,15709.0,NORM,100.0,records100/00000/00001_lr,NORM,NORM,NORM
1,2,13243.0,NORM,80.0,records100/00000/00002_lr,NORM,NORM,NORM
2,3,20372.0,NORM,100.0,records100/00000/00003_lr,NORM,NORM,NORM
3,4,17014.0,NORM,100.0,records100/00000/00004_lr,NORM,NORM,NORM
4,5,17448.0,NORM,100.0,records100/00000/00005_lr,NORM,NORM,NORM


In [18]:
data_one_scp_df["diagnostic_class"].unique()

array(['NORM', 'STTC', 'HYP', 'CD', 'MI'], dtype=object)

In [19]:
data_one_scp_df.loc[data_one_scp_df["diagnostic_class_numerical"] == "NORM", "diagnostic_class_numerical"] = 0
data_one_scp_df.loc[data_one_scp_df["diagnostic_class_numerical"] == "HYP", "diagnostic_class_numerical"] = 1
data_one_scp_df.loc[data_one_scp_df["diagnostic_class_numerical"] == "STTC", "diagnostic_class_numerical"] = 2
data_one_scp_df.loc[data_one_scp_df["diagnostic_class_numerical"] == "CD", "diagnostic_class_numerical"] = 3
data_one_scp_df.loc[data_one_scp_df["diagnostic_class_numerical"] == "MI", "diagnostic_class_numerical"] = 4
data_one_scp_df.head(20)
#'HYP', 'STTC', 'CD', 'NORM', 'MI

,ecg_id,patient_id,max_key,max_value,filename_lr,diagnostic_class,diagnostic_subclass,diagnostic_class_numerical
0,1,15709.0,NORM,100.0,records100/00000/00001_lr,NORM,NORM,0
1,2,13243.0,NORM,80.0,records100/00000/00002_lr,NORM,NORM,0
2,3,20372.0,NORM,100.0,records100/00000/00003_lr,NORM,NORM,0
3,4,17014.0,NORM,100.0,records100/00000/00004_lr,NORM,NORM,0
4,5,17448.0,NORM,100.0,records100/00000/00005_lr,NORM,NORM,0
5,6,19005.0,NORM,100.0,records100/00000/00006_lr,NORM,NORM,0
6,7,16193.0,NORM,100.0,records100/00000/00007_lr,NORM,NORM,0
7,9,18792.0,NORM,100.0,records100/00000/00009_lr,NORM,NORM,0
8,10,9456.0,NORM,100.0,records100/00000/00010_lr,NORM,NORM,0
9,11,11243.0,NORM,80.0,records100/00000/00011_lr,NORM,NORM,0


In [20]:

type(data_one_scp_df["ecg_id"].unique())



numpy.ndarray

In [21]:
print(data_one_scp_df["ecg_id"].unique().shape)
print(data_one_scp_df["patient_id"].unique().shape)

(19032,)
(16776,)


In [ ]:
X_train = pd.DataFrame
for i in range(data_one_scp_df["ecg_id"].unique().shape[0]):
    if len(X_train) == 0.7*data_one_scp_df["ecg_id"].unique().shape()[0]:
        print("Trainset done")
    
    else:
        X_train_interm = data_one_scp_df.iloc[i]
        
    

In [ ]:
data_one_scp_df["diagnostic_class_numerical"].unique()

In [22]:
prelim_featurevector = pd.DataFrame(data_one_scp_df).to_numpy()
# print(prelim_featurevector)
# print(type(prelim_featurevector))
# print(prelim_featurevector.shape)

In [24]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device is {DEVICE}")

Device is cuda


In [25]:
class ECGDataset(torch.utils.data.Dataset):
  def __init__(self, seqlen):
    self.data = prelim_featurevector

    self.ecg = None
    self.labels = None
    self.seqlen = seqlen
    
    for idx in tqdm.tqdm(range(self.data.shape[0])):

        path_wfdb_100 = os.path.join(path_data_csv, prelim_featurevector[idx][4])
        record_100 = wfdb.rdrecord(path_wfdb_100)
        signal_physical_100 = record_100.p_signal  

        ecg = torch.Tensor(signal_physical_100).to(DEVICE).unsqueeze(0) #.view(1,-1) (permute)
        
        #self.ecg.append(signal_physical_100)

        if self.ecg is None:
          self.ecg = ecg
        else:
          self.ecg = torch.concat((self.ecg, ecg), dim=0)
        #print(prelim_featurevector[idx][7])
        #label = torch.Tensor(prelim_featurevector[idx][-1]).type(torch.long).to(DEVICE)
        label = torch.Tensor([prelim_featurevector[idx][7]]).type(torch.long).to(DEVICE)
        #self.labels.append(prelim_featurevector[idx][2])
        
        if self.labels is None:
          self.labels = label
        else:
          self.labels = torch.concat((self.labels, label), dim=0)

    #self.ecg = torch.tensor(self.ecg)
    #self.labels = torch.tensor(self.labels)


  def __len__(self):
    return self.data.shape[0]
   # return len(self.data) - self.seqlen
  
  def __getitem__(self, idx):
    return self.ecg[idx, :, :], self.labels[idx]
    #return self.ecg[idx:(idx+self.seqlen)], self.labels[idx+self.seqlen]

In [26]:
dataset = ECGDataset(seqlen=4)

 65%|██████▌   | 12379/19032 [31:57<24:12,  4.58it/s]  

In [ ]:
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [ ]:
print(next(iter(dataloader)))

In [ ]:
batch = next(iter(dataloader))

In [ ]:
len(batch)

In [ ]:
batch[0].shape

In [ ]:
batch[1]

In [ ]:
batch[1].shape

In [ ]:
class LSTM1(nn.Module):
    def __init__(self, num_classes, input_size, hidden_size, num_layers, seq_length):
        super(LSTM1, self).__init__()
        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.seq_length = seq_length

        # Modify the input_size parameter to match the number of channels
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                            num_layers=num_layers, batch_first=True)
        self.fc_1 = nn.Linear(hidden_size, 128)
        self.fc = nn.Linear(128, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        h_0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c_0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        output, (hn, cn) = self.lstm(x, (h_0, c_0))
        hn = hn.view(-1, self.hidden_size)
        out = self.relu(hn)
        out = self.fc_1(out)
        out = self.relu(out)
        out = self.fc(out)
        return out

In [ ]:
class LSTM2(nn.Module):
    def __init__(self):
        super().__init__()

        self.lstm = nn.LSTM(input_size=12, hidden_size=128, batch_first=True)
        self.fc = nn.Linear(128,5)

    def forward(self, x):
        out, hidden = self.lstm(x)
        return self.fc(out[:,-1,:])



In [ ]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
 
        self.conv = nn.Sequential(
            nn.Conv2d(12, 24, kernel_size=(10,1), stride=(2,1)), nn.ReLU(),
            nn.Conv2d(24, 48, kernel_size=(10,1), stride=(2,1)), nn.ReLU(),
            nn.Conv2d(48, 96, kernel_size=(10,1), stride=(2,1)), nn.ReLU(),
            )
       
        self.lstm = nn.LSTM(input_size=96, hidden_size=128, batch_first=True)
 
        self.fc = nn.Linear(128, 5)
 
    def forward(self, x):
        x = x.view(-1, 12, 1000, 1)
        x = self.conv(x).view(-1,118,96)
        out, hidden = self.lstm(x)
        return self.fc(out[:,-1,:])
 
 
x = torch.rand((64, 1000, 12))
net = Net()
y = net(x)
print(x.shape)
print(y.shape)

In [ ]:
class Trainer:
  def __init__(self, network, loss_function):
    self.network = network
    self.loss_function = loss_function
    
    self.init_optimizer()
    self.init_scheduler()

  def init_optimizer(self):
    self.optim = torch.optim.Adam(self.network.parameters(), lr=0.0001)

  def init_scheduler(self):
    self.scheduler = torch.optim.lr_scheduler.ExponentialLR(self.optim, 0.95)

  def epoch(self, dataloader, training, epoch=0):
    # We want a dedicated TQDM bar, so we can set the description after each step
    bar = tqdm.tqdm(dataloader)
    
    # Tell the network whether we are training or evaluating (to disable DropOut)
    if training:
      self.network.train()
      name="train"
    else:
      self.network.eval()
      name="val"

    # This epoch starts
    total_loss = 0
    correct = 0
    cnt = 0

    # Iterate over the whole epoch
    for batch, labels in bar:
      # If we are training, zero out the gradients in the network
      if training:
        self.optim.zero_grad()

      # Do one forward pass
      lstm = self.network(batch)

      # Reshape labels for processing
      labels = labels.reshape

      # Calculcate the (BCE)-Loss
      loss = self.loss_function(lstm(batch), labels)

      # Sum the total loss
      total_loss += loss.item()

      # Count how many correct predictions we have (for accuracy)
      correct += torch.sum(torch.argmax(lstm, dim=1) == labels).item()

      # Count total samples processed
      cnt += batch.shape[0]

      # Update bar description

      bar.set_description(f"ep: {epoch:.0f} ({name}), loss: {1000.0*total_loss / cnt:.3f}, acc: {100.0*correct/cnt:.2f}%")

      # If we are training, do backward pass 
      if training:
          # Calculcate backward gradiesnts
          loss.backward()

          # Step the optimizer
          self.optim.step()

    return 1000.0 * total_loss / cnt, 100.0*correct/cnt

if __name__ == "__main__":
    dataset = ECGDataset(seqlen=4)
    dataloader = DataLoader(dataset, batch_size=32, shuffle=True)
    
    net = LSTM2().to(DEVICE)
    loss = torch.nn.CrossEntropyLoss()

    trainer = Trainer(net, loss)
    #trainer.epoch(dataloader, net, True)

    num_epochs = 10  # number of epochs
    for epoch in range(num_epochs):
        trainer.epoch(dataloader, net, True)


In [ ]:
len(dataset)

In [ ]:
torch.set_printoptions(profile="full")
#print(dataset.labels)
print(dataset.labels.size())

In [ ]:
for i in range (29):
    print(dataset[i][1])

In [ ]:
dataset[0][0].shape

In [ ]:
dataset[0][0]

In [ ]:
dataset[0][1]

In [ ]:
wfdb_data = []
for i in tqdm.tqdm(range(prelim_featurevector.shape[0])):
    #print(prelim_featurevector[i][4])
    path_wfdb_100 = os.path.join(path_data_csv,  prelim_featurevector[i][4])
    #path_wfdb_100
    record_100 = wfdb.rdrecord(path_wfdb_100)
    signal_physical_100 = record_100.p_signal
    
    wfdb_data.append(signal_physical_100)
    #print(wfdb_data)

In [ ]:
wfdb_data

In [ ]:
wfdb_data[0][0][0]

In [ ]:
wfdb_data[0]

In [ ]:
type(wfdb_data[0])

In [ ]:
wfdb_data_np = np.array(wfdb_data)
print(wfdb_data_np[0].shape)


In [ ]:
wfdb_data_np.shape[0]

In [ ]:
class DS():
    def __init__(self):
        pass

    def __len__(self):
        return self.wfdb_data_np.shape[0]
    
    def __getitem__(self, idx):
        return torch.Tensor(self.wfdb_data_np[idx]).to(DEVICE), 
                    torch.Tensor([prelim_featurevector_df[2].loc[idx]]).to(DEVICE).long()
    
ds = DS()
print(len(ds))

In [ ]:
class WaveData(Dataset):
    def __init__(self):
        # data loading
        pass
    
    def __getitem__(self, index):
        # allows indexing
        pass

    def __len__(self):
        # len(dataset)
        pass

In [ ]:
prelim_featurevector_df["wfdb"] = wfdb_data_np.tolist()

In [ ]:
prelim_featurevector_df

In [ ]:
prelim_featurevector_wfdb_np = prelim_featurevector_df.to_numpy()

In [ ]:
prelim_featurevector_wfdb_np.shape

In [ ]:
prelim_featurevector_wfdb_np

In [ ]:
#wfdb_data_np = wfdb_data_np.reshape(19683, -1)
prelim_featurevector_wfdb = np.concatenate((prelim_featurevector, wfdb_data_np[:,:,:].reshape((19683, -1))), axis=1)

In [ ]:
prelim_featurevector_wfdb = np.hstack((prelim_featurevector, wfdb_data_np))

In [ ]:
prelim_featurevector_wfdb.shape

In [ ]:
print(path_wfdb)
path_wfdb_100 = os.path.join(path_wfdb,  prelim_featurevector[0][4])
path_wfdb_100

In [ ]:
record = wfdb.rdrecord(path_wfdb_100)
wfdb.plot_wfdb(record=record, title='Trial, ECG, Ending .hea') 
display(record.__dict__)

In [ ]:
signal_physical = record.p_signal
print(signal_physical)
signal_physical.shape